In [1]:
import glob
import os

import geopandas as gpd
import nivapy3 as nivapy
import pandas as pd
from sqlalchemy import text
from tqdm.notebook import tqdm

# Create PostGIS database for TEOTIL3

To streamline the workflow for TEOTIL3, I propose migrating all key datasets from Oracle to the Hub's PostGIS database. The code in this notebook creates a new schema named `teotil3` within the `general` database and loads relevant datasets to PostGIS.

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(admin=True)

Username:  ········
Password:  ········


Connection successful.


## 1. Create schema and set permissions

In [3]:
# Drop schema if it already exists - USE WITH CAUTION!
sql = "DROP SCHEMA teotil3 CASCADE"
eng.execute(sql)

/tmp/ipykernel_7190/2819351708.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  eng.execute(sql)


In [4]:
# Create schema
sql = "CREATE SCHEMA IF NOT EXISTS teotil3"
eng.execute(sql)

In [5]:
# Grant "ReadOnly" privileges to default Jovyan user
sql_list = [
    "GRANT USAGE ON SCHEMA teotil3 TO jovyan",
    "GRANT SELECT ON ALL TABLES IN SCHEMA teotil3 TO jovyan",
    "ALTER DEFAULT PRIVILEGES IN SCHEMA teotil3 GRANT SELECT ON TABLES TO jovyan",
]
for sql in sql_list:
    eng.execute(sql)

## 2. Spatial tables

In [6]:
spatial_data_fold = f"/home/jovyan/shared/common/teotil3/core_data"
teo_gpkg = os.path.join(spatial_data_fold, "tidied", f"teotil3_data.gpkg")
reg_gdf = gpd.read_file(teo_gpkg, layer=f"regines", driver="GPKG")
reg_gdf.head()

,regine,vassom,a_cat_poly_km2,upstr_a_km2,ospar_region,komnr_2014,fylnr_2014,komnr_2015,fylnr_2015,komnr_2016,...,a_sea_km2,a_upland_km2,a_urban_km2,a_wood_km2,ar50_tot_a_km2,a_cat_land_km2,a_lake_nve_km2,runoff_mm/yr,q_cat_m3/s,geometry
0,001.10,001,1.442790,0.00,Skagerrak,0101,01,0101,01,0101,...,0.281940,0.000000,0.000000,0.849188,0.849201,1.160850,0.000000,592,0.02178,"POLYGON ((297006.830 6543966.950, 297169.290 6..."
1,001.1A1,001,1.432479,777.90,Skagerrak,0101,01,0101,01,0101,...,0.000067,0.004615,0.000000,1.377476,1.430189,1.432412,0.043955,620,0.02814,"POLYGON ((297505.440 6543157.790, 297543.100 6..."
2,001.1A20,001,0.340160,777.90,Skagerrak,0101,01,0101,01,0101,...,0.000045,0.000000,0.000000,0.303492,0.340114,0.340114,0.000000,594,0.00640,"POLYGON ((297770.368 6543429.036, 297787.114 6..."
3,001.1A2A,001,17.647822,58.96,Skagerrak,0101,01,0101,01,0101,...,0.000000,0.467374,0.131585,15.030746,17.647822,17.647822,0.186340,637,0.35623,"POLYGON ((299678.370 6544460.320, 299667.220 6..."
4,001.1A2B,001,41.298255,41.30,Skagerrak,0101,01,0101,01,0101,...,0.000000,2.250799,0.161524,29.798394,41.298255,41.298255,7.344123,637,0.83362,"POLYGON ((303353.460 6552989.330, 303341.620 6..."


In [7]:
table_name = f"regines"

nivapy.da.gdf_to_postgis(
    reg_gdf,
    table_name,
    "teotil3",
    eng,
    f"{table_name}_spidx",
    create_pk=False,
    index=False,
    if_exists="replace",
)

sql = (
    f"ALTER TABLE teotil3.{table_name} "
    f"ADD CONSTRAINT {table_name}_pk "
    f'PRIMARY KEY ("regine")'
)
eng.execute(sql)

WARNING! The dataframe contains mixed geometries:
  {'MultiPolygon', 'Polygon'}
These will be cast to "Multi" type. If this is not what you want, consider using gdf.explode() first


## 3. Non-spatial tables

### 3.1. Basic tables

In [8]:
# point_source_locations
sql = text("DROP TABLE IF EXISTS teotil3.point_source_locations CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.point_source_locations "
    "( "
    "  site_id text NOT NULL, "
    "  name text NOT NULL, "
    "  sector text NOT NULL, "
    "  type text NOT NULL, "
    "  year integer NOT NULL, "
    "  site_geom geometry(Point, 25833) NOT NULL, "
    "  outlet_geom geometry(Point, 25833) NOT NULL, "
    "  PRIMARY KEY (site_id, year), "
    "  CONSTRAINT site_unique UNIQUE (site_id, name, sector, type, year, outlet_geom) "
    ")"
)
eng.execute(sql)

# input_param_definitions
sql = text("DROP TABLE IF EXISTS teotil3.input_param_definitions CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.input_param_definitions "
    "( "
    "  in_par_id integer NOT NULL, "
    "  oracle_par_id integer, "
    "  name text NOT NULL, "
    "  unit text NOT NULL, "
    "  comment text, "
    "  PRIMARY KEY (in_par_id), "
    "  CONSTRAINT in_par_unique UNIQUE (in_par_id, name, unit) "
    ")"
)
eng.execute(sql)

# output_param_definitions
sql = text("DROP TABLE IF EXISTS teotil3.output_param_definitions CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.output_param_definitions "
    "( "
    "  out_par_id integer NOT NULL, "
    "  name text NOT NULL, "
    "  unit text NOT NULL, "
    "  PRIMARY KEY (out_par_id), "
    "  CONSTRAINT out_par_unique UNIQUE (out_par_id, name, unit) "
    ")"
)
eng.execute(sql)

# input_output_param_conversion
sql = text("DROP TABLE IF EXISTS teotil3.input_output_param_conversion CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.input_output_param_conversion "
    "( "
    "  in_par_id integer NOT NULL, "
    "  out_par_id integer NOT NULL, "
    "  factor numeric NOT NULL, "
    "  PRIMARY KEY (in_par_id, out_par_id), "
    "  CONSTRAINT in_par_id_fkey FOREIGN KEY (in_par_id) "
    "      REFERENCES teotil3.input_param_definitions (in_par_id), "
    "  CONSTRAINT out_par_id_fkey FOREIGN KEY (out_par_id) "
    "      REFERENCES teotil3.output_param_definitions (out_par_id) "
    ")"
)
eng.execute(sql)

# point_source_values
sql = text("DROP TABLE IF EXISTS teotil3.point_source_values CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.point_source_values "
    "( "
    "  site_id text NOT NULL, "
    "  in_par_id integer NOT NULL, "
    "  year integer NOT NULL, "
    "  value numeric NOT NULL, "
    "  PRIMARY KEY (site_id, in_par_id, year), "
    "  CONSTRAINT site_id_fkey FOREIGN KEY (site_id, year) "
    "      REFERENCES teotil3.point_source_locations (site_id, year), "
    "  CONSTRAINT in_par_id_fkey FOREIGN KEY (in_par_id) "
    "      REFERENCES teotil3.input_param_definitions (in_par_id) "
    ")"
)
eng.execute(sql)

# spredt_inputs
sql = text("DROP TABLE IF EXISTS teotil3.spredt_inputs CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.spredt_inputs "
    "( "
    "  komnr text NOT NULL, "
    "  in_par_id integer NOT NULL, "
    "  year integer NOT NULL, "
    "  value numeric NOT NULL, "
    "  PRIMARY KEY (komnr, in_par_id, year), "
    "  CONSTRAINT in_par_id_fkey FOREIGN KEY (in_par_id) "
    "      REFERENCES teotil3.input_param_definitions (in_par_id) "
    ")"
)
eng.execute(sql)

# agri_inputs
sql = text("DROP TABLE IF EXISTS teotil3.agri_inputs CASCADE")
eng.execute(sql)

sql = text(
    "CREATE TABLE teotil3.agri_inputs "
    "( "
    "  loss_type text NOT NULL, "
    "  year integer NOT NULL, "
    "  regine text NOT NULL, "
    "  agriculture_totn_kg numeric NOT NULL, "
    "  agriculture_background_totn_kg numeric NOT NULL, "
    "  agriculture_din_kg numeric NOT NULL, "
    "  agriculture_background_din_kg numeric NOT NULL, "
    "  agriculture_ton_kg numeric NOT NULL, "
    "  agriculture_background_ton_kg numeric NOT NULL, "
    "  agriculture_totp_kg numeric NOT NULL, "
    "  agriculture_background_totp_kg numeric NOT NULL, "
    "  agriculture_tdp_kg numeric NOT NULL, "
    "  agriculture_background_tdp_kg numeric NOT NULL, "
    "  agriculture_tpp_kg numeric NOT NULL, "
    "  agriculture_background_tpp_kg numeric NOT NULL, "
    "  agriculture_toc_kg numeric NOT NULL, "
    "  agriculture_background_toc_kg numeric NOT NULL, "
    "  agriculture_ss_kg numeric NOT NULL, "
    "  agriculture_background_ss_kg numeric NOT NULL, "
    "  PRIMARY KEY (loss_type, year, regine), "
    "  CONSTRAINT regine_fkey FOREIGN KEY (regine) "
    "      REFERENCES teotil3.regines (regine) "
    ")"
)
eng.execute(sql)

In [9]:
# input_param_definitions
csv_path = r"../../data/postgis_input_parameter_definitions.csv"
df = pd.read_csv(csv_path)
df.to_sql(
    "input_param_definitions", eng, schema="teotil3", index=False, if_exists="append"
)

# output_param_definitions
csv_path = r"../../data/postgis_output_parameter_definitions.csv"
df = pd.read_csv(csv_path)
df.to_sql(
    "output_param_definitions",
    eng,
    schema="teotil3",
    index=False,
    if_exists="append",
)

# input_output_param_conversion
csv_path = r"../../data/postgis_input_output_param_conversion.csv"
df = pd.read_csv(csv_path)
df.dropna(how='any', inplace=True)
df.to_sql(
    "input_output_param_conversion",
    eng,
    schema="teotil3",
    index=False,
    if_exists="append",
)

110

### 3.2. Additional tables

In [10]:
# data_fold = r"../../data"
# data_year = 2022

# # key: value = table_name: primary key
# csv_dict = {
#     "ar50_artype_classes.csv": "artype",
#     "lake_residence_times_10m_dem.csv": "vatnLnr",
#     "offshore_hierarchy.csv": "regine",
#     "regine_retention_transmission_10m_dem.csv": "regine",
#     "spatially_static_background_coefficients.csv": "variable",
#     "spatially_variable_background_coefficients.csv": "regine",
#     "spatiotemporally_variable_background_coefficients.csv": "regine",
#     "vassdragsomrader_ospar_regions.csv": "vassom",
# }

# for fname, pk_col in csv_dict.items():
#     table_name = fname[:-4] + f"_{data_year}"
#     fpath = os.path.join(data_fold, fname)
#     df = pd.read_csv(fpath)
#     df.to_sql(table_name, eng, schema="teotil3", index=False, if_exists="replace")

#     sql = (
#         f"ALTER TABLE teotil3.{table_name} "
#         f"ADD CONSTRAINT {table_name}_pk "
#         f'PRIMARY KEY ("{pk_col}")'
#     )
#     eng.execute(sql)

## 4. HBV modelled discharge from NVE

In [11]:
# Period of data to upload (i.e. final year in datasets; for each of these,
# a whole time series from 1990 will be added. The data_delivery_year is 
# final_year + 1)
st_yr, end_yr = 2016, 2022

# Whether to replace the entire table or append to it
if_exists = "replace"

In [12]:
# Folder containing modelled data
data_fold = r"/home/jovyan/shared/common/teotil3/nve_hbv_data"

years = range(st_yr, end_yr + 1)
df_list = []
for year in years:
    search_path = os.path.join(data_fold, f"RID_{year}", "hbv_*.var")
    flist = glob.glob(search_path)

    # Get number of days between 1990 and year of interest
    days = len(pd.date_range(start="1990-01-01", end="%s-12-31" % year, freq="D"))

    for fpath in flist:
        name = os.path.split(fpath)[1]
        vassom = name.split("_")[1][-7:-4]

        df = pd.read_csv(
            fpath, delim_whitespace=True, header=None, names=["date", "flow_m3/s"]
        )
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d/1200")
        df["vassom"] = vassom
        df["data_supply_year"] = year + 1
        df = df[["data_supply_year", "vassom", "date", "flow_m3/s"]]

        # Check st, end and length
        assert df["date"].iloc[0] == pd.Timestamp(
            "1990-01-01"
        ), "Series does not start on 01/01/1990."
        assert df["date"].iloc[-1] == pd.Timestamp("%s-12-31" % year), (
            "Series does not end on 31/12/%s." % year
        )
        assert len(df) == days, "Unexpected length for new series."

        df_list.append(df)

df = pd.concat(df_list, axis="rows")
assert df.duplicated(["data_supply_year", "vassom", "date"], keep=False).sum() == 0

print(f"{len(df)/1e6:.1f} million rows to insert.")

20.0 million rows to insert.


In [13]:
%%time

# The databasse can't cope with writing 20 M rows directly from pandas
# Instead, manually split the dataframe into chunks and write one
# at a time
chunk_size = 100000

table_name = "nve_hbv_discharge"

if if_exists == "replace":
    # Replace with empty table
    df.iloc[:0].to_sql(
        table_name,
        eng,
        schema="teotil3",
        index=False,
        if_exists=if_exists,
    )

# Write chunks in append mode
chunks = [df[i : i + chunk_size] for i in range(0, df.shape[0], chunk_size)]
for chunk in tqdm(chunks):
    chunk.to_sql(
        table_name,
        eng,
        schema="teotil3",
        index=False,
        if_exists="append",
        method="multi",
    )

sql = (
    f"ALTER TABLE teotil3.{table_name} "
    f"ADD CONSTRAINT {table_name}_pk "
    f"PRIMARY KEY (data_supply_year, vassom, date)"
)
eng.execute(sql)

  0%|          | 0/201 [00:00<?, ?it/s]

CPU times: user 31min 48s, sys: 43.6 s, total: 32min 32s
Wall time: 38min 55s
